# Importaciones

In [ ]:
import sys
!{sys.executable} -m pip install gdown --quiet

In [ ]:
# Importamos las librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
import pickle
from google.colab import files
import gdown
from sklearn.decomposition import PCA

In [ ]:
# Descargamos el Dataset de Train/Val desde el enlace
url = "https://drive.google.com/uc?export=download&id=16By7bbG6lG6PkVF-17uDLVPN7fMH98j-"
# Leemos el Dataset
df = pd.read_csv(url)

# Descargamos el Dataset de Test desde el enlace
url = "https://drive.google.com/uc?export=download&id=1hSvTbDT2Je4ILWOmIP_bWyxhu9ViaUqF"
# Leemos el Dataset
df_test = pd.read_csv(url)

In [ ]:
# Descargamos las medias generacionales saltando la primera fila (que solo contiene los índices de generaciones)
url = "https://drive.google.com/uc?export=download&id=13E8bSkQ-jX313whRzVTuNdQ1jaKhEHQ-"
# Leemos el Dataset
df_medias = pd.read_csv(url, header=None, skiprows=1)

# Visualización inicial

In [ ]:
# Ver tamaño
df.shape

In [ ]:
df_test.shape

In [ ]:
# Ver tipos de datos
df.dtypes

In [ ]:
df.head()

In [ ]:
df_test.head()

In [ ]:
# Nulos por columna
df.isnull().sum()

# Preprocesamiento del Dataframe

In [ ]:
# Eliminamos la columna ID porque no nos sirve de nada
df = df.drop(columns=['ID'])
df_test = df_test.drop(columns=['ID'])

In [ ]:
# Nombrar las filas del df de medias
df_medias.index = ["celtas", "iberos", "fenicios", "griegos", "italicos"]
# Transponer para que cada generación sea una fila
df_medias_T = df_medias.T.reset_index(drop=True)
df_medias_T.columns = ["celtas", "iberos", "fenicios", "griegos", "italicos"]
df_medias_T.head()

# Análisis de la distribución de la variable objetivo

In [ ]:
print(df["distancia_G1"].describe())

El 75% de los valores de la variable están entre 1 y 25, el otro 25% están entre 25 y 117

In [ ]:
plt.hist(df["distancia_G1"], bins=30, color='skyblue', edgecolor='black')
plt.title("Distribución de distancia_G1")
plt.xlabel("Valor")
plt.ylabel("Frecuencia")
plt.show()

In [ ]:
sns.boxplot(x=df["distancia_G1"], color='lightgreen')
plt.title("Boxplot de distancia_G1")
plt.show()

Podemos ver claramente que la variable objetivo (distancia_G1) tiene una distribución muy asimétrica, hay muchisimos más valores bajos que valores altos.




# Preparación del Dataframe para los entrenamientos

In [ ]:
# Definir variables X e y

# Dataframe principal
X = df[["celtas", "iberos", "fenicios", "griegos", "italicos"]]
y = df["distancia_G1"]

# Dataframe de test
X_test = df_test[["celtas", "iberos", "fenicios", "griegos", "italicos"]]
y_test = df_test["distancia_G1"]

In [ ]:
# Dividimos el Dataframe
# Train 80%, Val 20%
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("Train:", X_train.shape[0], "filas")
print("Validación:", X_val.shape[0], "filas")

In [ ]:
# Inicializar el escalador
scaler = StandardScaler()

# Escalamos solo con los datos de entrenamiento
X_train_scaled = scaler.fit_transform(X_train)

# Transformamos validación y test con el mismo escalador
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Transformamos también las medias generacionales que se encuentran en df_medias_T
medias_scaled = scaler.transform(df_medias_T)

# Definir y entrenar el Modelo LinearRegression

In [ ]:
model = LinearRegression()
model.fit(X_train_scaled, y_train)

In [ ]:
y_pred = model.predict(X_val_scaled)

## Ver los resultados de predicción en los datos de validación

In [ ]:
r2 = r2_score(y_val, y_pred)
mae = mean_absolute_error(y_val, y_pred)
rmse = np.sqrt(mean_squared_error(y_val, y_pred))

print("Resultados en validación:")
print(f"R²   = {r2:.3f}")
print(f"MAE  = {mae:.3f}")
print(f"RMSE = {rmse:.3f}")

Teniendo en cuenta que la variable objetivo es muy asimetrica, estos resultados tan malos nos hacen sospechar que los valores más altos (outliers), que estan menos respresentados y muy alejados del grueso de valores, pueden estar dificultando el aprendizaje del modelo lineal.

In [ ]:
# Crear máscaras para valores bajos y altos
low_mask = y_val <= 25.15
high_mask = y_val > 25.15

# Calcular MAE por segmento
mae_low = mean_absolute_error(y_val[low_mask], y_pred[low_mask])
mae_high = mean_absolute_error(y_val[high_mask], y_pred[high_mask])

# Mostrar resultados
print("Errores segmentados por rango de distancia_G1:")
print(f"MAE en valores bajos (≤ 25): {mae_low:.3f}")
print(f"MAE en valores altos (> 25): {mae_high:.3f}")

Las sospechas eran ciertas, en los valores por debajo de 25 el modelo predice ligeramente mal, sin embargo en los que están por encima el modelo predice muy mal (tres veces peor), esto como dijimos antes, pasa porque los valores altos son menos frecuentes y muy diferentes en escala, por lo que el modelo lineal (que asume una relación proporcional constante) no puede ajustarse bien.

# Definir y entrenar el Modelo GradientBoosting

Con la idea de obtener unos resultados decentes vamos a optar por un modelo Gradient Boosting, el cual nos ofrece las siguientes ventajas:

- Maneja relaciones no lineales entre variables.

- Funciona bien con rangos de valores muy distintos.

- Se puede obtener muy buena precisión si se ajustan bien los hiperparámetros.

In [ ]:
gb_model = GradientBoostingRegressor(
    n_estimators=100,     # número de árboles
    learning_rate=0.1,    # velocidad de aprendizaje
    max_depth=3,          # profundidad máxima de cada árbol
    random_state=42
)

Debido a que el tamaño del Dataframe no es enorme (tiene 560 filas para entrenar), un modelo muy profundo o con muchos árboles puede sobreajustar, asi que no conviene pasarse usando hiparametros demasiado grandes.

Además por la misma razón, conviene escalar los datos, como ya hemos hecho antes, para evitar modelos extremadamente complejos.




In [ ]:
gb_model.fit(X_train_scaled, y_train)

In [ ]:
y_pred = gb_model.predict(X_val_scaled)

## Ver los resultados de predicción en los datos de validación

In [ ]:
r2 = r2_score(y_val, y_pred)
mae = mean_absolute_error(y_val, y_pred)
rmse = np.sqrt(mean_squared_error(y_val, y_pred))

print("Resultados Gradient Boosting en validación:")
print(f"R²   = {r2:.3f}")
print(f"MAE  = {mae:.3f}")
print(f"RMSE = {rmse:.3f}")

Ahora podemos ver como los resultados han mejorado enormemente, con una mayor explicación de la variabilidad y un menor error promedio

In [ ]:
low_mask = y_val <= 25.15
high_mask = y_val > 25.15

mae_low = mean_absolute_error(y_val[low_mask], y_pred[low_mask])
mae_high = mean_absolute_error(y_val[high_mask], y_pred[high_mask])

print("Errores segmentados por rango de distancia_G1:")
print(f"MAE en valores bajos (≤ 25): {mae_low:.3f}")
print(f"MAE en valores altos (> 25): {mae_high:.3f}")

Ahora el modelo captura bien tanto los valores bajos como los altos

## Búsqueda de los mejores hiperparámetros

Ya que hemos visto que el modelo funciona bien, vamos a hacer una búsqueda de hiperparámetros para Gradient Boosting usando GridSearchCV, lo cuál nos permitirá encontrar la combinación que maximice el rendimiento del modelo

In [ ]:
# Definir el modelo base
gb = GradientBoostingRegressor(random_state=42)

# Definir el marco de hiperparámetros a probar
param_grid = {
    "n_estimators": [100, 200, 300],
    "learning_rate": [0.05, 0.1, 0.2],
    "max_depth": [2, 3, 4],
    "min_samples_split": [2, 5, 10]
}

# Configurar GridSearchCV
grid_search = GridSearchCV(
    estimator=gb,
    param_grid=param_grid,
    scoring="neg_mean_absolute_error",  # buscamos minimizar MAE
    cv=5,                               # validación cruzada 5-fold
    n_jobs=-1
)

In [ ]:
grid_search.fit(X_train_scaled, y_train)

In [ ]:
# Mejor combinación de hiperparámetros
print("Mejores hiperparámetros encontrados:")
print(grid_search.best_params_)

In [ ]:
# Evaluar en validación
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_val_scaled)

In [ ]:
r2 = r2_score(y_val, y_pred)
mae = mean_absolute_error(y_val, y_pred)
rmse = np.sqrt(mean_squared_error(y_val, y_pred))

print("Resultados con los mejores hiperparámetros:")
print(f"R²   = {r2:.3f}")
print(f"MAE  = {mae:.3f}")
print(f"RMSE = {rmse:.3f}")

Con el ajuste de hiperparámetros hemos reducido el error promedio general considerablemente

In [ ]:
low_mask = y_val <= 25.15
high_mask = y_val > 25.15

mae_low = mean_absolute_error(y_val[low_mask], y_pred[low_mask])
mae_high = mean_absolute_error(y_val[high_mask], y_pred[high_mask])

print("Errores segmentados por rango de distancia_G1:")
print(f"MAE en valores bajos (≤ 25): {mae_low:.3f}")
print(f"MAE en valores altos (> 25): {mae_high:.3f}")

Además hemos conseguido reducir todavía más el error promedio en valores bajos

# Probar el Modelo en el conjunto de Test

In [ ]:
y_test_pred = best_model.predict(X_test_scaled)

In [ ]:
r2_test = r2_score(y_test, y_test_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))

print("Resultados en Test:")
print(f"R²   = {r2_test:.3f}")
print(f"MAE  = {mae_test:.3f}")
print(f"RMSE = {rmse_test:.3f}")

In [ ]:
low_mask_test = y_test <= 25.15
high_mask_test = y_test > 25.15

mae_low_test = mean_absolute_error(y_test[low_mask_test], y_test_pred[low_mask_test])
mae_high_test = mean_absolute_error(y_test[high_mask_test], y_test_pred[high_mask_test])

print("Errores segmentados en Test:")
print(f"MAE en valores bajos (≤ 25): {mae_low_test:.3f}")
print(f"MAE en valores altos (> 25): {mae_high_test:.3f}")

En base a los buenos resultados obtenidos podemos interpretar lo siguiente:

- El modelo generaliza correctamente ya que las métricas bajaron un poco (como es normal), pero siguen muy altas
- No hay sobreajuste ya que si lo hubiera el R² se hubiera desplomado
- Hay consistencia en los valores altos, ya que el MAE apenas cambió con respecto a los resultados de validación

# Serialización del Modelo

In [ ]:
# Crear un diccionario con los objetos a guardar
modelo_completo = {
    "model": best_model,
    "scaler": scaler
}

In [ ]:
# Guardar en Colab
archivo_pickle = "gradient_boosting_model.pkl"
with open(archivo_pickle, "wb") as f:
    pickle.dump(modelo_completo, f)

# Descargar automáticamente
files.download(archivo_pickle)

print("Modelo y preprocesadores guardados correctamente")

# Cargar modelo .pkl

In [ ]:
url = "https://drive.google.com/uc?export=download&id=1yB477t_Ese8qFf9xQoMg1fXuo9EXQoDH"
output = "gradboost_model_genes.pkl"

# Descargar desde Drive
gdown.download(url, output, quiet=False)

# Cargar el modelo y preprocesadores
with open(output, "rb") as f:
    data_cargada = pickle.load(f)

modelo_cargado = data_cargada["model"]
scaler_cargado = data_cargada["scaler"]

print("\nModelo y preprocesadores cargados correctamente desde Google Drive.")

In [ ]:
# Probar modelo cargado en el conjunto de test
y_test_pkl = modelo_cargado.predict(X_test_scaled)

r2_test = r2_score(y_test, y_test_pkl)
mae_test = mean_absolute_error(y_test, y_test_pkl)
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pkl))

print("Resultados en Test:")
print(f"R²   = {r2_test:.3f}")
print(f"MAE  = {mae_test:.3f}")
print(f"RMSE = {rmse_test:.3f}")

# PCA

In [ ]:
# PCA
pca = PCA(n_components=2)
medias_pca = pca.fit_transform(medias_scaled)

In [ ]:
plt.figure(figsize=(7, 6))

# G1 = referencia ancestral
plt.scatter(medias_pca[0, 0], medias_pca[0, 1],
            color="red", s=150, marker="X", label="Generación 1 (ancestral)")

# Otras generaciones
plt.scatter(medias_pca[1:, 0], medias_pca[1:, 1],
            color="gray", label="Otras generaciones", alpha=0.7)

# Etiquetas G1–G9 (centradas sobre el punto)
for i in range(medias_pca.shape[0]):
    plt.text(medias_pca[i, 0], medias_pca[i, 1],
             f"G{i+1}",
             fontsize=9, ha="center", va="center",
             bbox=dict(facecolor="white", alpha=0.6, edgecolor="none", pad=1))

plt.xlabel("Componente principal 1")
plt.ylabel("Componente principal 2")
plt.title("PCA: Generaciones")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Tomamos un individuo del test
i = 96  # por ejemplo, el individuo número X
individuo = X_test_scaled[i].reshape(1, -1)  # mantener formato 2D

# Lo proyectamos al mismo espacio PCA
individuo_pca = pca.transform(individuo)

In [ ]:
plt.figure(figsize=(7, 6))

# Generación ancestral (G1)
plt.scatter(medias_pca[0, 0], medias_pca[0, 1],
            color="red", s=150, marker="X", label="Generación 1 (ancestral)")

# Otras generaciones
plt.scatter(medias_pca[1:, 0], medias_pca[1:, 1],
            color="gray", label="Otras generaciones", alpha=0.7)

# Individuo de test
plt.scatter(individuo_pca[0, 0], individuo_pca[0, 1],
            color="blue", s=100, label=f"Individuo test #{i}")

# Etiquetas de generaciones
for j in range(medias_pca.shape[0]):
    plt.text(medias_pca[j, 0], medias_pca[j, 1],
             f"G{j+1}", fontsize=9, ha="center", va="center",
             bbox=dict(facecolor="white", alpha=0.6, edgecolor="none", pad=1))

plt.xlabel("Componente principal 1")
plt.ylabel("Componente principal 2")
plt.title("PCA: Generaciones y un individuo de test")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Guardar en Colab el PCA entrenado
archivo_pca = "pca.pkl"
with open(archivo_pca, "wb") as f:
    pickle.dump(pca, f)

# Descargar automáticamente
files.download(archivo_pca)

print("PCA guardado correctamente")

In [ ]:
# Guardar las coordenadas PCA de las generaciones
df_medias_pca = pd.DataFrame(medias_pca, columns=["PC1", "PC2"])
df_medias_pca.to_csv("medias_pca.csv", index=False)

# Descargar automáticamente
files.download("medias_pca.csv")

print("Coordenadas PCA de las generaciones guardadas correctamente")